## Radiometric Terrain Correction of Sentinel-1 data using a chosen DEM

The accuracy of Radiometric Terrain Corrected (RTC) products is strongly affected by the resolution of the input DEM. Although Sentinel-1 RTC is the only product currently available on the Planetary Computer, the open-source package [sarsen](https://github.com/bopen/sarsen) enables to perform corrections on any DEM product.

In this tutorial, you will learn how to perform (i) geometric and (ii) radiometric terrain corrections on the Sentinel-1 GRD product using sarsen. The comparison at the end of this notebook demonstrates that the RTC computed by sarsen is consistent with the RTC already available on the Planetary Computer.

As an example, we use data convering the South-of-Redmond region (Seattle, US).
Steps:
- Download the Sentinel-1 GRD
- Download the 10-meter DEM
- Compute the GTC using sarsen
- Compute the RTC using sarsen
- Compare the GTC to the RTC
- Compare the RTC computed using sarsen to the RTC already available on the Planetery Computer 

### Introduction
The typical side-looking SAR system acquires data with uniform sampling in azimuth and slant range, where the azimuth and range represents the time when a given target is acquired and the absolute sensor-to-target distance, respectively.
Because of this, the near range appears compressed with respect to the far range. Furthermore, any deviation of the target elevation from a smooth geoid results in additional local geometric and radiometric distortions known as foreshortening, layover and shadow.
- Radar foreshortening: Terrain surfaces sloping towards the radar appear shortened relative to those sloping away from the radar.
- Radar layover: It's an extreme case of foreshortening occurring when the terrain slope is greater than the angle of the incident signal.  
- Radar shadows: They occur when ground points at the same azimuth but different slant ranges are aligned in the direction of the line-of-sight. This is usually due to a back slope with an angle steeper than the viewing angle. When this happens, the radar signal never reaches the farthest points, and thus there is no measurement, meaning that this lack of information is unrecoverable.


### Geometric Terrain Correction
The GRD Sentinel-1 product has been processed applying a geometric correction. Such a correction removes the near-range compression effect, converting the data from slant-range to ground-range coordinates with uniform sampling.  
Similarly, we will correct the distorsions due to the target elevation in the GTC product. While the GRD products can be computed without a digital elevation model (DEM), the GTC processing requires a DEM to associate the ground points to the image points. 


### Radiometric Terrain Correction
Terrain variations do not affect the position of a given point on the Earth's surface only, but they also affect the brightness of the radar return. The sarsen radiometric terrain correction compensates for the backscatter modulation generated by the topography of the scene. This produces a more uniform backscatter image, emphasizing the radiometric differences of the terrain.

<hr style="border:2px solid blue"> </hr>

Dependecies: sarsen 

In [ ]:
!pip install -q sarsen

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
import os
import tempfile

# enable the `.rio` accessor
import rioxarray  # noqa: F401
import xarray as xr

from sarsen import apps

### Processing definition

In [ ]:
# create a temporary directory where to store downloaded data
tmp_dir = tempfile.gettempdir()

# DEM path
dem_path = os.path.join(tmp_dir, "South-of-Redmond-10m.tif")

# path to Sentinel-1 input product in the Planetary Computer
product_folder = "GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048"  # noqa: E501

# band to be processed
measurement_group = "IW/VV"

tmp_dir

###  DEM of South-of-Redmond (Seattle, US)

In [ ]:
import adlfs
import planetary_computer
import pystac_client
import stackstac

#### Area of interest definition

In [ ]:
lon, lat = [-121.95, 47.04]
buffer = 0.2
bbox = [lon - buffer, lat - buffer, lon + buffer, lat + buffer]

#### DEMs discover

Here we use the DEM with a 10-meter ground sample distance (GDS) available on the Planetary Computer. Note that any DEM supported by GDAL/Proj can be used.

Using `pystac_client` we can search the Planetary Computer's STAC endpoint for items matching our query parameters.  
As multiple DEMs acquired at different times are available in this area, we select the DEMs with 10-meter GDS and perform the average of the remaining DEMs along the time dimension.

**Note:** It may take some time to retrieve the DEM.

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())

Here we load the data into an xarray DataArray using stackstac.

In [ ]:
# select DEMs with resolution 10 meters
items_high_res = [
    planetary_computer.sign(item).to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

dem_raster_all = stackstac.stack(items_high_res, bounds=bbox).squeeze()
dem_raster_all

#### Average the DEMs along time

In [ ]:
dem_raster_geo = dem_raster_all.compute()
if "time" in dem_raster_geo.dims:
    dem_raster_geo = dem_raster_geo.mean("time")
_ = dem_raster_geo.rio.set_crs(dem_raster_all.rio.crs)

#### Convert the DEM in UTM coordinates

In order to facilitate the comparison between the RTC computed by sarsen with the RTC available on the Planetery Computer, here we convert the DEM in UTM.

In [ ]:
# find the UTM zone and project in UTM
t_srs = dem_raster_geo.rio.estimate_utm_crs()
dem_raster = dem_raster_geo.rio.reproject(t_srs, resolution=(10, 10))

# crop DEM to our area of interest and save it
dem_corners = dict(x=slice(565000, 594000), y=slice(5220000, 5190000))

dem_raster = dem_raster.sel(**dem_corners)
dem_raster.rio.to_raster(dem_path)
dem_raster

In [ ]:
dem_raster.plot()
_ = plt.title("DEM in UTM coordinates")

### Retrieve Sentinel-1 GRD

**Note:** It may take some time to retrieve the Sentinel-1 GRD. In the future, it will be possible to access data via [fsspec](https://filesystem-spec.readthedocs.io/en/latest/) without having to download data locally.

In [ ]:
grd_account_name = "sentinel1euwest"
grd_storage_container = "s1-grd"
grd_token = planetary_computer.sas.get_token(
    grd_account_name, grd_storage_container
).token

grd_product_folder = f"{grd_storage_container}/{product_folder}"

grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(f"{grd_product_folder}/manifest.safe")

In [ ]:
grd_local_path = os.path.join(tmp_dir, product_folder)
grd_fs.get(grd_product_folder, grd_local_path, recursive=True)
!ls -d {grd_local_path}

### Processing

#### GTC

Here we compute the geometric terrain correction.

Input parameters:
- `product_urlpath`: product path
- `measurement_group`: band to be processed in the form {swath}/{polarization} (see [xarray-sentinel](https://pypi.org/project/xarray-sentinel/) for more details)
- `dem_urlpath`: path to the input DEM. sarsen supports all DEMs supported by GDAL/Proj for ECEF-translation. 
- `interp_method`: interpolation method, sarsen supports all interpolation methods supported by [xarray.Dataset.interp](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.interp.html)
- `chunks`: dask chunks
- `output_urlpath`: output path

The output is the input SAR image resampled on DEM coordinates. 

**Note:** It may take some time to compute the GTC.

In [ ]:
gtc = apps.backward_geocode_sentinel1(
    product_urlpath=grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    output_urlpath=os.path.basename(product_folder) + ".10m.GTC.tif",
)

In [ ]:
_ = gtc.plot(vmax=0.4)

#### RTC
sarsen implements the radiometric terrain-correction [Gamma Flattening](https://ieeexplore.ieee.org/document/5752845) algorithm.

Input parameters:
- `correct_radiometry`: if `True`, it activates the Gamma Flattening correction.
- `grouping_area_factor`: is a tuple of floats greater than 1. The default is `(1, 1)`.

**Note**: The `grouping_area_factor` can be increased (i) to speed up the processing or (ii) when the input DEM resolution is low. The Gamma Flattening usually works properly if the pixel size of the input DEM is much smaller than the pixel size of the input Sentinel-1 product. Otherwise, the output may have radiometric distortions. This problem can be avoided by increasing the `grouping_area_factor`. Be aware that `grouping_area_factor` too high may degrade the final result.

**Note:** As the RTC genaration step loads the data into the memory, it may take serveral minutes. The performances will be improved in the next releases of sarsen.

In [ ]:
rtc = apps.backward_geocode_sentinel1(
    grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    correct_radiometry=True,
    output_urlpath=os.path.basename(product_folder) + ".10m.RTC.tif",
    grouping_area_factor=(3, 3),
)

### Comparison between GTC and RTC

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

gtc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")
plt.title("GTC")


rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")
plt.title("RTC")


plt.tight_layout()

### Comparison between sarsen RTC and Planetary Computer RTC


Retrieve the Sentinel-1 RTC

**Note:** It may take some time to retrieve the RTC.

In [ ]:
rtc_account_name = "sentinel1euwestrtc"
rtc_storage_container = "sentinel1-grd-rtc"
rtc_token = planetary_computer.sas.get_token(
    rtc_account_name, rtc_storage_container
).token

rtc_product_folder = f"{rtc_storage_container}/{product_folder}"

rtc_fs = adlfs.AzureBlobFileSystem(rtc_account_name, credential=rtc_token)
rtc_fs.ls(rtc_product_folder)

In [ ]:
rtc_local_path = os.path.join(tmp_dir, rtc_product_folder)
rtc_fs.get(f"{rtc_product_folder}", rtc_local_path, recursive=True)
!ls -d {rtc_local_path}

In [ ]:
rtc_pc = xr.open_dataarray(
    rtc_local_path + "/measurement/iw-vv.rtc.tiff", cache=False
).drop("band")
rtc_pc = rtc_pc.sel(dem_corners)
rtc_pc

#### Plot sarsen RTC and Planetary Computer RTC

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

rtc_pc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")
plt.title("Planetary Computer RTC")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")
plt.title("sarsen RTC")

plt.tight_layout()